In [2]:
%load_ext autoreload
%autoreload 1

In [55]:
%aimport exv2.experiment_results 

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import glasbey



pd.set_option('display.max_columns', None)
sns.set_theme(rc={'figure.figsize':(12, 6)})
sns.set_context("paper")
sns.set_style("whitegrid")

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

palette = glasbey.create_block_palette(
    [4, 3, 3, 2, 2],
    colorblind_safe=True,
    cvd_severity=90
)
sns.set_palette(palette)

dep_focus = ["baseline_vanilla_full","monolith_feature_monolith_full","serverless_feature_serverless_full"]
ms_focus = ["baseline_vanilla_full","obs_feature_object-storage_full","norec_feature_norecommendations_full"]
lable_names = {
    "baseline_vanilla_full": "Microservices",
    'jvm_jvm-impoove_full': "Runtime Improvement",
    'monolith_feature_monolith_full': "Monolith",
    'norec_feature_norecommendations_full' : "Service Reduction",
    'obs_feature_object-storage_full' : "Infrastructure Service Replacement",
    'serverless_feature_serverless_full' : "Serverless",
    'serverless_incl_knative': "Serverless (+KNative)"
}


left = "exp_scale_pausing"
right = "exp_scale_rampup"

In [4]:
exr = exv2.experiment_results.ExperimentResults("data/2024-07-*", load_stats_history=True) # type: ignore
RUN_VARS = exv2.experiment_results.ExperimentResults.RUN_VARS # type: ignore

assert("loadgenerator" not in (list(exr.pods['name'].unique())))
assert("unkown" not in (list(exr.pods['instance'].unique())))


/Users/b/projects/teastore_experiments/exv2/experiment_results.py:182: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_pods = pd.concat(


In [73]:



failures = exr.stats_history_aggregated[exr.stats_history_aggregated["exp_workload"].isin([left,right])].groupby(["exp_branch","exp_workload"])[["rq","frq"]].sum()
failures["Failure Rate"] = 100*failures["frq"] / failures["rq"]
failures = failures.unstack()
failures["fr"] = failures["Failure Rate"].apply(lambda x: f'{x[left]:>2.2} - {x[right]:>2.2f}', axis=1)
failures = failures.droplevel(1, axis=1).reset_index()[["exp_branch","fr"]]

failures


,exp_branch,fr
0,baseline_vanilla_full,3.5 - 11.51
1,jvm_jvm-impoove_full,2.3 - 0.03
2,monolith_feature_monolith_full,0.89 - 41.80
3,norec_feature_norecommendations_full,1.9 - 1.78
4,obs_feature_object-storage_full,1.4 - 69.96
5,serverless_feature_serverless_full,5.1 - 9.31


In [7]:
latency = exr.stats_history_aggregated[exr.stats_history_aggregated["exp_workload"].isin([left,right])].groupby(["exp_branch","exp_workload"])[["p50", "p95"]].mean().unstack()
latency = latency / 1000
latency["p50_diff"] = latency["p50"].apply(lambda x: f'{x[left]:>2.2f} - {x[right]:>2.2f}', axis=1)
latency["p95_diff"] = latency["p95"].apply(lambda x: f'{x[left]:>2.2f} - {x[right]:>2.2f}', axis=1)

# get the range from a moderate workload (shaped) to a stress workload (rampup)

latency = latency.droplevel(1, axis=1).reset_index()
latency = latency[["exp_branch","p50_diff","p95_diff"]]
latency

,exp_branch,p50_diff,p95_diff
0,baseline_vanilla_full,0.06 - 6.31,0.17 - 16.37
1,jvm_jvm-impoove_full,0.02 - 1.36,0.10 - 12.42
2,monolith_feature_monolith_full,0.01 - 23.23,0.04 - 42.78
3,norec_feature_norecommendations_full,0.06 - 2.61,0.20 - 8.36
4,obs_feature_object-storage_full,0.07 - 17.36,0.20 - 27.93
5,serverless_feature_serverless_full,0.75 - 4.68,1.76 - 15.38


In [77]:
# AWS cost-model

serverless_price = 0.0000166667  # based on aws lambda price per GB-s  (frankfurt)
memory_second_price = 0.00511 / 1024 /60 # $/MBs based on AWS nfragate memory price per hour (frankfurt) 
vCPU_second_price = 0.04656 / 60 # $/vCPU based on AWS nfragate memory price per hour (frankfurt) 

pod_configuration = {
    "teastore-recommender": {"cpu": 2600, "memory": 1332},
    "teastore-webui": {"cpu": 1300, "memory": 1950},
    "teastore-image": {"cpu": 1300, "memory": 1950},
    "teastore-auth": {"cpu": 585, "memory": 1332},
    'teastore-registry':{"cpu": 1000, "memory": 1024}, # not set by default ....
    'teastore-persistence':{"cpu": 1000, "memory": 1024}, # not set by default ....
    'teastore-db':{"cpu": 1000, "memory": 1024}, # not set by default ....
    "teastore-all": {"cpu":1950, "memory":2663},
    "auth": {"cpu": 500, "memory": 500},
}

import numpy as np
# we calculate the cost for each pod based on the configuration and the time it was running
def calc_request_based_billing(row):
    if row["type"] == "pod":
        conf = pod_configuration[row["pod_name"]] 
        return conf["memory"] * memory_second_price + np.ceil(conf["cpu"]/1000) * vCPU_second_price
    elif row["type"] == "function":
        return 500*serverless_price
    

def calc_usage_based_billing(row):
    if row["type"] == "pod":
        return row["memory_usage"] * memory_second_price + np.ceil(row["cpu_usage"]) * vCPU_second_price
    elif row["type"] == "function":
        return row["memory_usage"]*serverless_price

pods = exr.pods[exr.pods["namespace"] == "tea-bench"]
pods["pod_name"] = pods["name"].apply(lambda x: "-".join(x.split("-")[0:2]))

pods["type"] = pods["pod_name"].apply(lambda x: "pod" if x.startswith("teastore") else "function" if x.startswith("auth") else "infra")
# ignore infra pods for now
pods = pods[pods["type"].isin(["pod","function"])]
pods_usage = pods.groupby(exr.RUN_VARS+["run_time","name","pod_name","type"])[["memory_usage","cpu_usage"]].sum().reset_index()

pods_usage["requested_cost"] = pods_usage.apply(calc_request_based_billing, axis=1)
pods_usage["used_cost"] = pods_usage.apply(calc_usage_based_billing, axis=1)

pods_mean_cost = pods_usage.groupby(exr.RUN_VARS)[["requested_cost","used_cost"]].sum().reset_index().groupby(["exp_branch","exp_workload"])[["requested_cost","used_cost"]].mean().reset_index()

requests = exr.stats.groupby(["exp_branch","exp_workload"])[["Request Count","Failure Count"]].sum().reset_index() # total request count
requests["rq"] = requests["Request Count"] - requests["Failure Count"]

pods_mean_cost_per_request = pods_mean_cost.merge(requests[["exp_branch","exp_workload","rq"]], on=["exp_branch","exp_workload"])
pods_mean_cost_per_request["requested_cost_per_r"] = (pods_mean_cost_per_request["requested_cost"] / pods_mean_cost_per_request["rq"]) * 100 * 1000 # convert to mili cents
pods_mean_cost_per_request["used_cost_per_r"] = (pods_mean_cost_per_request["used_cost"] / pods_mean_cost_per_request["rq"]) * 100 * 1000 # convert to mili cents


pods_cost_comp = pods_mean_cost_per_request[pods_mean_cost_per_request["exp_workload"] == left].merge(pods_mean_cost_per_request[pods_mean_cost_per_request["exp_workload"] == right], on="exp_branch", suffixes=("_left","_right"))
pods_cost_comp["requested_cost"] = pods_cost_comp.apply(lambda x: f'{x["requested_cost_left"]:>2.2f} - {x["requested_cost_right"]:>2.2f}', axis=1)
pods_cost_comp["used_cost"] = pods_cost_comp.apply(lambda x: f'{x["used_cost_left"]:>2.2f} - {x["used_cost_right"]:>2.2f}', axis=1)
pods_cost_comp["used_cost_per_r"] = pods_cost_comp.apply(lambda x: f'{x["used_cost_per_r_left"]:>2.2f} - {x["used_cost_per_r_right"]:>2.2f}', axis=1)
pods_cost_comp = pods_cost_comp[["exp_branch","requested_cost","used_cost","used_cost_per_r"]]
pods_cost_comp

,exp_branch,requested_cost,used_cost,used_cost_per_r
0,baseline_vanilla_full,0.58 - 0.84,0.27 - 0.41,24.01 - 0.26
1,jvm_jvm-impoove_full,0.58 - 0.82,0.27 - 0.40,23.11 - 0.10
2,monolith_feature_monolith_full,0.16 - 0.26,0.08 - 0.11,10.10 - 0.77
3,norec_feature_norecommendations_full,0.69 - 0.86,0.28 - 0.41,24.98 - 0.10
4,obs_feature_object-storage_full,0.69 - 0.74,0.28 - 0.28,24.15 - 2.70
5,serverless_feature_serverless_full,4.08 - 4.60,0.67 - 0.94,63.49 - 0.53


In [74]:
main_table = latency.merge(failures, on="exp_branch").merge(pods_cost_comp, on="exp_branch")
main_table = main_table[main_table["exp_branch"].isin(dep_focus)]
main_table["exp_branch"] = main_table["exp_branch"].map(lable_names)
main_table.columns = pd.MultiIndex.from_tuples([("","Feature"),("Latency", "p50 [s]"), ("Latency", "p95 [s]"), ( "Failure Rate [\%] ",""), ("Costs", "Total Cost [$\$$]"), ("Costs", "Consumed [$\$$]"), ("Costs", "Per Request [\textcent /1000]")])
main_table

Latency               Failure Rate [\%]   \
         Feature       p50 [s]       p95 [s]                      
0  Microservices   0.06 - 6.31  0.17 - 16.37        3.5 - 11.51   
2       Monolith  0.01 - 23.23  0.04 - 42.78       0.89 - 41.80   
5     Serverless   0.75 - 4.68  1.76 - 15.38         5.1 - 9.31   

              Costs                                               
  Total Cost [$\$$] Consumed [$\$$] Per Request [textcent /1000]  
0       0.58 - 0.84     0.27 - 0.41                 24.01 - 0.26  
2       0.16 - 0.26     0.08 - 0.11                 10.10 - 0.77  
5       4.09 - 4.60     0.67 - 0.94                 63.49 - 0.53

In [75]:
with open("figs/master_table.tex", "w") as f:
    f.write(main_table.to_latex(index=False, escape=False, column_format="lcccccc", multicolumn_format="c"))